In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [15]:
from ff_espn_api import League

In [3]:
# Varberg-Lindquist League
league_id = 1086928
year = 2018
swid = input();
espn_s2 = input();
clear_output()
league = League(league_id, year, espn_s2, swid)

In [16]:
# Mounds View League
league_id = 200140
year = 2019
swid = input();
espn_s2 = input();
clear_output()
league = League(league_id, year, espn_s2, swid)

In [39]:
league = league2

In [41]:
#### Preseason
# Replacement points dict for all positions
repl_points = {'QB': 263,
              'WR': 100,
              'TE': 83,
              'K': 116,
              'RB': 82,
              'D/ST': 93}
#
nick = league.teams[7]
data_name = 'FFA_Projections_2019.csv'
pre_projections = pd.read_csv(data_name)
pre_projections = pre_projections.set_index('name_espn')
for team in league.teams:
    for player in team.roster:
        if player.name in pre_projections.index:
            player.pre_proj_ppg = pre_projections.loc[player.name, 'Points']
        else:
            player.pre_proj_ppg = repl_points[player.position]
            print(player.name + ' not found so I guess I\'ll give him ' + str(player.pre_proj_ppg))

Jack Doyle not found so I guess I'll give him 83
Jalen Richard not found so I guess I'll give him 82
Patriots D/ST not found so I guess I'll give him 93
Mason Crosby not found so I guess I'll give him 116
Kenny Stills not found so I guess I'll give him 100
Adam Humphries not found so I guess I'll give him 100
Mike Badgley not found so I guess I'll give him 116
Broncos D/ST not found so I guess I'll give him 93
Albert Wilson not found so I guess I'll give him 100
DaeSean Hamilton not found so I guess I'll give him 100
Jake Elliott not found so I guess I'll give him 116
Willie Snead IV not found so I guess I'll give him 100
Chase Edmonds not found so I guess I'll give him 82
Mohamed Sanu not found so I guess I'll give him 100
Adam Vinatieri not found so I guess I'll give him 116
Mike Evans not found so I guess I'll give him 100
Deebo Samuel not found so I guess I'll give him 100
C.J. Anderson not found so I guess I'll give him 82
Eagles D/ST not found so I guess I'll give him 93
Chiefs D

In [5]:
#### Midseason
mv_or_vl = 'vl'
if mv_or_vl == 'mv':
    pass
else:
    # change this later so we don't need week
    NUM_WEEKS = 11
    WEEK = 4
    NUM_PLAY_WEEKS = 6
    wins   = [1,1,1,3,4,1,2,3]
    losses = [3,3,3,1,0,3,2,1]
# Change this later but let's make up wins and losses
for team,cnt in zip(league.teams, wins):
    team.wins = cnt
for team,cnt in zip(league.teams, losses):
    team.losses = cnt

In [8]:
ppg_list = []
proj_ppg_list = []
for team in league.teams:
    ppg = np.round(sum(team.scores[0:WEEK])/WEEK,2)
    team.ppg = ppg
    team.proj_ppg = ppg
    std_dev = round(np.std(team.scores[0:WEEK]),1)
    team.std_dev = std_dev
    team.proj_std_dev = std_dev
    ppg_list.append(team.ppg)
    proj_ppg_list.append(team.proj_ppg)

In [9]:
# create schedule: list of weeks, each week is list of matchups
schedule = []
for game in range(WEEK,NUM_WEEKS):
    week_sched = []
    for team in league.teams: 
        if((team not in week_sched) & (team.schedule[game] not in week_sched)):
            week_sched.append(team) 
            week_sched.append(team.schedule[game])
    schedule.append(week_sched)

In [11]:
for team in league.teams:
    team.sim_fst   = 0
    team.sim_semi   = 0
    team.sim_final  = 0
    team.sim_champ  = 0
#### Start iterations
## Sim Regular Season
iters = 30000
for iterations in range(0, iters):
    for team in league.teams:
        team.sim_wins       = team.wins
        team.sim_losses     = team.losses
        team.sim_weeks      = np.random.normal(team.proj_ppg, team.proj_std_dev, NUM_WEEKS-WEEK)
        team.sim_points_for = sum(team.sim_weeks)
    for week in range(0,len(schedule)):
        for game_num in range(0,len(schedule[week]),2):
            team1 = schedule[week][game_num]
            team2 = schedule[week][game_num+1]
            if(team1.sim_weeks[week] > team2.sim_weeks[week]):
                team1.sim_wins   += 1
                team2.sim_losses += 1
            else:
                team1.sim_losses += 1
                team2.sim_wins   += 1
    # Sort by sim_points_for, then sort by sim_wins to get standing
    standings = sorted(league.teams, key=lambda x: x.sim_points_for, reverse=True)
    standings = sorted(standings, key=lambda x: x.sim_wins, reverse=True)
    ## Sim Playoffs
    if mv_or_vl == 'vl':
        for team in standings[0:7]:
                team.sim_fst +=1
        for team in standings[0:7]:
            team.sim_play = np.random.normal(2*team.proj_ppg, 2**(1/2)*team.proj_std_dev, int(NUM_PLAY_WEEKS/2))
        fst_rd = 0
        semi_1 = standings[0]
        if standings[1].sim_play[fst_rd] > standings[6].sim_play[fst_rd]:
            semi_2 = standings[1]
        else:
            semi_2 = standings[6]
        if standings[2].sim_play[fst_rd] > standings[5].sim_play[fst_rd]:
            semi_3 = standings[2]
        else:
            semi_3 = standings[5]
        if standings[3].sim_play[fst_rd] > standings[4].sim_play[fst_rd]:
            semi_4 = standings[3]
        else:
            semi_4 = standings[4]
        semi_1.sim_semi += 1
        semi_2.sim_semi += 1
        semi_3.sim_semi += 1
        semi_4.sim_semi += 1
        semi = 1
        if semi_1.sim_play[semi] > semi_2.sim_play[semi]:
            final_1 = semi_1
        else:
            final_1 = semi_4
        if semi_2.sim_play[semi] > semi_3.sim_play[semi]:
            final_2 = semi_2
        else:
            final_2 = semi_3
        final_1.sim_final += 1
        final_2.sim_final += 1
        champ = 2
        if final_1.sim_play[champ] > final_2.sim_play[champ]:
            champ = final_1
        else:
            champ = final_2
        champ.sim_champ += 1

In [13]:
# Create dataframe with teams data
tm_nm_list = []
abbrev = []
wins = []
losses = []
fst_list = []
semi_list = []
final_list = []
champ_list = []
for team in standings:
    tm_nm_list.append(team.team_name)
    abbrev.append(team.team_abbrev)
    wins.append(team.wins)
    losses.append(team.losses)
    fst_list.append(np.round(team.sim_fst/iters*100, 1))
    semi_list.append(np.round(team.sim_semi/iters*100, 1))
    final_list.append(np.round(team.sim_final/iters*100, 1))
    champ_list.append(np.round(team.sim_champ/iters*100, 1))
teamsdf = pd.DataFrame({'Owner': abbrev})
teamsdf = teamsdf.set_index('Owner')
teamsdf['Team'] = tm_nm_list
teamsdf['Wins'] = wins
teamsdf['Losses'] = losses
teamsdf['proj_ppg'] = proj_ppg_list
teamsdf['Playoffs'] = fst_list
teamsdf['Semifinals'] = semi_list
teamsdf['Finals'] = final_list
teamsdf['Champion'] = champ_list
teamsdf.sort_values(by='Champion', ascending=False)

,Team,Wins,Losses,proj_ppg,Playoffs,Semifinals,Finals,Champion
Owner,,,,,,,,
nath,Kelce Kapistrant,2,2,135.50,100.0,94.4,53.5,32.2
Drew,U of Cal - Berk Oski the Bear,3,1,106.50,100.0,90.8,51.1,28.7
Tmmy,Reigning Champion,3,1,105.50,100.0,96.0,47.8,21.6
Tom,Kirk and Dak LLC,4,0,100.25,100.0,94.0,41.6,17.0
Zach,Winning is just my Forte,1,3,108.25,81.8,9.4,2.6,0.3
Jord,I Have Adrian ... Again,1,3,136.25,78.7,6.6,1.5,0.2
Tim,Team Tim,1,3,128.50,87.2,6.7,1.6,0.1
Nick,Baker Mayfield bro'd me,1,3,130.25,52.3,2.1,0.3,0.0


In [108]:
from datetime import date
if mv_or_vl == 'vl':
    name = 'VL_predictions_week_'+str(WEEK)
    pd.DataFrame.to_csv(teamsdf,name)